In [3]:
import cv2
import pytesseract
import numpy as np

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Change this path as per your setup

def preprocess_image(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to remove noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply threshold to get a binary image
    _, binary_img = cv2.threshold(blurred, 150, 255, cv2.THRESH_BINARY_INV)

    return binary_img

def extract_numbers(image_path):
    # Preprocess the image
    processed_img = preprocess_image(image_path)

    # Use pytesseract to extract text
    extracted_text = pytesseract.image_to_string(processed_img, config='--psm 6')

    # Extract only digits
    numbers = ''.join(filter(str.isdigit, extracted_text))

    return numbers

if __name__ == "__main__":
    # Path to your image
    image_path = 'image2.jpg'  # Replace with the actual path to your image

    # Extract numbers from the image
    numbers = extract_numbers(image_path)
    
    # Print the extracted numbers
    if numbers:
        print(f"Extracted Numbers: {numbers}")
    else:
        print("No numbers found.")


Extracted Numbers: 87729444727232


In [4]:
import cv2
import pytesseract
import numpy as np

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Change this path as per your setup

def preprocess_image(image_path):
    """Preprocesses an image for OCR.

    Args:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray: Preprocessed image.
    """

    # Load the image
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to remove noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Perform morphological operations to refine the image
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours and filter for text regions
    contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    text_regions = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h
        if 0.2 <= aspect_ratio <= 7:
            text_regions.append((x, y, w, h))

    # Crop the text regions from the image
    cropped_images = []
    for x, y, w, h in text_regions:
        cropped_img = img[y:y+h, x:x+w]
        cropped_images.append(cropped_img)

    return cropped_images

def extract_numbers(image_path):
    """Extracts numbers from an image.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Extracted numbers.
    """

    cropped_images = preprocess_image(image_path)

    extracted_numbers = []
    for img in cropped_images:
        # Use pytesseract with custom configuration for number extraction
        text = pytesseract.image_to_string(img, config='--psm 6 --oem 3')
        numbers = ''.join(filter(str.isdigit, text))
        extracted_numbers.append(numbers)

    # Combine extracted numbers from multiple regions
    final_numbers = ''.join(extracted_numbers)

    return final_numbers

if __name__ == "__main__":
    image_path = 'image2.jpg'  # Replace with the actual path to your image

    try:
        numbers = extract_numbers(image_path)
        if numbers:
            print(f"Extracted Numbers: {numbers}")
        else:
            print("No numbers found.")
    except Exception as e:
        print(f"Error extracting numbers: {e}")

Extracted Numbers: 2441592084084710443727777757244065354444


In [ ]:
import io
import os
from google.cloud import vision
from google.cloud.vision_v1 import types

# Set the environment variable for Google Application Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"path_to_your_service_account_json.json"

def extract_text_from_image(image_path):
    """Extract text from image using Google Cloud Vision API"""
    
    # Initialize a Google Vision client
    client = vision.ImageAnnotatorClient()

    # Load the image from file
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
        image = vision.Image(content=content)

    # Perform text detection
    response = client.text_detection(image=image)
    texts = response.text_annotations

    # Extract the detected text
    extracted_text = texts[0].description if texts else "No text found"

    if response.error.message:
        raise Exception(f'{response.error.message}')

    return extracted_text

if __name__ == "__main__":
    # Path to the image
    image_path = 'image2.jpg'  # Replace with the actual path to your image

    # Extract text from the image
    extracted_text = extract_text_from_image(image_path)

    # Print the extracted text
    print(f"Extracted Text:\n{extracted_text}")


In [ ]:
import os
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
import io

# Set up your Azure credentials
subscription_key = '1c64e220-7176-4ad8-902c-0aab8d6d62d8'
endpoint = 'https://mycomputeronvision.cognitiveservices.azure.com/'

# Authenticate the client
client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

def extract_text_from_image(image_path):
    """Extract text from image using Azure Computer Vision API"""

    # Read the image file
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    # Use the Computer Vision API to extract text
    ocr_result = client.read_in_stream(io.BytesIO(image_data), raw=True)

    # Get the operation location (URL) for the result
    operation_location = ocr_result.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]

    # Poll for the result
    while True:
        result = client.get_read_result(operation_id)
        if result.status not in [OperationStatusCodes.running, OperationStatusCodes.not_started]:
            break

    # Extract the text from the result
    if result.status == OperationStatusCodes.succeeded:
        text = ""
        for result in result.analyze_result.read_results:
            for line in result.lines:
                text += line.text + " "
        return text.strip()
    else:
        return "OCR failed or no text found."

if __name__ == "__main__":
    # Path to your image
    image_path = 'image2.jpg'  # Replace with the actual path to your image

    # Extract text from the image
    extracted_text = extract_text_from_image(image_path)
    
    # Print the extracted text
    print(f"Extracted Text:\n{extracted_text}")


In [8]:
import boto3
import io

# Initialize the Amazon Rekognition client
client = boto3.client('rekognition', region_name='ap-southeast-2')

def extract_text_from_image(image_path):
    """Extract text from image using Amazon Rekognition"""

    # Read the image file
    with open(image_path, 'rb') as image_file:
        image_bytes = image_file.read()

    # Call Amazon Rekognition to detect text
    response = client.detect_text(
        Image={'Bytes': image_bytes}
    )

    # Extract the detected text
    detected_text = ""
    for item in response['TextDetections']:
        detected_text += item['DetectedText'] + " "

    return detected_text.strip()

if __name__ == "__main__":
    # Path to your image
    image_path = 'image2.jpg'  # Replace with the actual path to your image

    # Extract text from the image
    extracted_text = extract_text_from_image(image_path)
    
    # Print the extracted text
    print(f"Extracted Text:\n{extracted_text}")


NoCredentialsError: Unable to locate credentials